In [1]:
#数据加载
import  pandas as pd
import numpy as np
data = pd.read_csv('./family_data.csv',index_col='family_id')
print(data)

           choice_0  choice_1  choice_2  choice_3  choice_4  choice_5  \
family_id                                                               
0                52        38        12        82        33        75   
1                26         4        82         5        11        47   
2               100        54        25        12        27        82   
3                 2        95         1        96        32         6   
4                53         1        47        93        26         3   
...             ...       ...       ...       ...       ...       ...   
4995             16         1        66        33        18        70   
4996             88        66        20        17        26        54   
4997             32        66        54        17        27        21   
4998             67        92         4        17        53        77   
4999             13        11        25        80        88        40   

           choice_6  choice_7  choice_8  choice_9 

In [2]:
def get_penalty(n,choice):
    penalty = None
    if choice == 0:
        penalty = 0
    if choice == 1:
        penalty = 50
    if choice == 2:
        penalty = 50+9*n
    if choice == 3:
        penalty = 100+9*n
    if choice == 4:
        penalty = 200+9*n
    if choice == 5:
        penalty = 200+18*n
    if choice == 6:
        penalty = 300+18*n
    if choice == 7:
        penalty = 300+36*n
    if choice == 8:
        penalty = 400+36*n
    if choice == 9:
        penalty = 500+(36+199)*n
    if choice >9:
        penalty = 500 + (36 + 398) * n
    return penalty

In [3]:
#常量定义
N_DAYS = 100 #安排的天数
N_FAMILY = 5000 #家庭id的个数
MIN_OCCUPANCY = 125 #最小承载量
MAX_OCCUPANCY = 300#最大承载量


#计算pcost_mat ,每个家庭，在什么时候（day 0-99） 访问时的penalty
#大小  5000 * 100的矩阵
pcost_mat = np.full(shape=(N_FAMILY,100),fill_value=999999)

for f in range(N_FAMILY):
    #家庭成员数
    f_num = data.loc[f,'n_people']
    #print(f_num)
    #对于第f个家庭，初始化pcost = other choice下的penalty
    pcost_mat[f,:] = get_penalty(f_num,10)
    #计算choice0-9的penalty
    for choice in range(10):#choice 0-9
        temp = data.loc[f][choice] #choice的天数
        penalty = get_penalty(f_num,choice)
        pcost_mat[f,temp-1] = penalty

print(pcost_mat)


[[2236 2236 2236 ... 2236 2236 2236]
 [2236 2236 2236 ... 2236 2236 2236]
 [1802 1802 1802 ... 1802 1802    0]
 ...
 [3104 3104  616 ... 3104 3104 3104]
 [ 390 2670 2670 ... 2670 2670 2670]
 [2236 2236 2236 ... 2236 2236 2236]]


In [4]:
pcost_mat.shape

(5000, 100)

In [5]:
#计算accountting_penalty矩阵，前一天的的参观人数，当天的参观人数
acost_mat = np.zeros(shape=(MAX_OCCUPANCY+1,MAX_OCCUPANCY+1),dtype=np.float64)
for i in range(acost_mat.shape[0]):#当天安排的人数
    for j in range(acost_mat.shape[1]):
        diff = abs(i-j)
        acost_mat[i][j] = max(0,(i -125)/400  * i **(0.5 + diff/50.0))

print(acost_mat)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [4.16316072e+15 3.71482922e+15 3.31477861e+15 ... 7.46610759e+00
  8.36716954e+00 9.37697794e+00]
 [4.79555148e+15 4.27883100e+15 3.81778713e+15 ... 8.43020770e+00
  7.52185316e+00 8.43020770e+00]
 [5.52415954e+15 4.92860244e+15 4.39725208e+15 ... 9.51970597e+00
  8.49339085e+00 7.57772228e+00]]


In [6]:
FAMILY_SIZE = data['n_people'].values
FAMILY_SIZE

array([4, 4, 3, ..., 6, 5, 4], dtype=int64)

In [7]:
#DESIRED 代表每个家庭CHOICE
DESIRED = data.values[:,:-1]-1
DESIRED

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]], dtype=int64)

In [8]:
DESIRED.shape

(5000, 10)

In [9]:
#使用线性规划
from ortools.linear_solver import pywraplp
def solveLP():
    #线性规划 优化器
    solver =pywraplp.Solver('AssignmentProblem',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    x = {} #family_id 在第j天是否参观
    #每一天有哪些家庭
    candidates = [[] for x in range(N_DAYS)]

    for i in range(N_FAMILY): #family_id
        for j in DESIRED[i,:]:#family_id的choice
            candidates[j].append(i) #在第j天，有第i个family参观
            #定义变量[i][j] i代表family_id j代表第j天参观
            x[i,j] = solver.BoolVar('x[%i,%i]' %(i,j))
    #print(x)

    #每天参观的人数
    daily_occupancy = [solver.Sum([x[i,j]*FAMILY_SIZE[i] for i in candidates[j]]) for j in range(N_DAYS)]

    #每个家庭，在10个choice中出现的总数
    family_presence = [solver.Sum(x[i,j] for j in DESIRED[i,:]) for i in range(N_FAMILY)]

    #定义目标函数， preference_cost部分
    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] for i in range(N_FAMILY) for j in DESIRED[i,:]])

    #满足preference_cost 最小
    solver.Minimize(preference_cost)

    #人为增加新的约束 constraints
    for j in range(N_DAYS-1):
        #当前人数不超过前一天人数+25
        solver.Add(daily_occupancy[j] - daily_occupancy[j+1] <=25)
        solver.Add(daily_occupancy[j+1] - daily_occupancy[j] <= 25)

    #每个家庭都在10个choice中出现1次
    for i in range(N_FAMILY):
        solver.Add(family_presence[i] ==1)

    #每天访问人数约束
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j] >=MIN_OCCUPANCY)
        solver.Add(daily_occupancy[j] <=MAX_OCCUPANCY)

    result = solver.Solve()
    # 查看解
    temp = [(i, j, x[i,j].solution_value()) for i in range(N_FAMILY) for j in DESIRED[i,:] if x[i, j].solution_value() > 0]

    #得到参观日的安排
    df = pd.DataFrame(temp,columns=['family_id','day','result'])
    print(solver.Objective().Value())
    return  df

In [10]:
result = solveLP()


73702.31696428571


In [11]:
print('result:',result['result'].value_counts())

result: 1.000000    4301
1.000000     627
0.750000       2
0.666667       2
0.666667       2
            ... 
0.750000       1
0.400000       1
0.400000       1
0.400000       1
0.947917       1
Name: result, Length: 144, dtype: int64


In [12]:
result

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5068,4995,15,1.0
5069,4996,87,1.0
5070,4997,31,1.0
5071,4998,91,1.0


In [13]:
#设置阈值
THRS = 0.99
assigned_df = result[result.result >THRS]
assigned_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5068,4995,15,1.0
5069,4996,87,1.0
5070,4997,31,1.0
5071,4998,91,1.0


In [14]:
#没有安排的，不为0和1
unassigned_df = result[(result.result <THRS) & (result.result > 1-THRS)]
unassigned_df

,family_id,day,result
59,59,38,0.25
60,59,14,0.75
241,240,32,0.75
242,240,56,0.25
264,262,31,0.50
...,...,...,...
4983,4912,8,0.40
4985,4914,38,0.60
4986,4914,43,0.40
5033,4961,53,0.75


In [15]:
#计算每天访问的人数(根据assigned_df)
assigned_df['family_size'] = FAMILY_SIZE[assigned_df.family_id]
assigned_df

D:\python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5068,4995,15,1.0,4
5069,4996,87,1.0,2
5070,4997,31,1.0,6
5071,4998,91,1.0,5


In [16]:
#按照day的方式聚合
occupancy = assigned_df.groupby('day').family_size.sum().values
occupancy

array([290, 271, 294, 293, 263, 242, 223, 247, 273, 297, 288, 292, 275,
       250, 238, 272, 292, 292, 271, 248, 223, 244, 264, 291, 292, 296,
       273, 249, 234, 251, 278, 283, 252, 235, 205, 184, 202, 233, 253,
       231, 210, 183, 204, 229, 247, 281, 256, 223, 204, 198, 222, 248,
       255, 223, 208, 185, 173, 196, 219, 198, 174, 141, 124, 121, 149,
       170, 178, 158, 136, 123, 125, 135, 158, 185, 158, 135, 121, 125,
       142, 167, 186, 167, 138, 120, 128, 155, 174, 203, 177, 152, 130,
       122, 130, 158, 178, 158, 128, 125, 122, 124], dtype=int64)

In [17]:

#更新阈值范围
min_occupancy = np.array([max(0,MIN_OCCUPANCY-x) for x in occupancy])#因为已经接待了线性规划求解数量的人，所以这里需要减去那些，剩下的为最小值

print(min_occupancy)

max_occupancy = np.array([MAX_OCCUPANCY-x for x in occupancy])
max_occupancy

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 4 0 0 0 0 0 2 0 0 0 0
 0 0 4 0 0 0 0 0 0 5 0 0 0 0 0 0 0 3 0 0 0 0 0 0 3 1]


array([ 10,  29,   6,   7,  37,  58,  77,  53,  27,   3,  12,   8,  25,
        50,  62,  28,   8,   8,  29,  52,  77,  56,  36,   9,   8,   4,
        27,  51,  66,  49,  22,  17,  48,  65,  95, 116,  98,  67,  47,
        69,  90, 117,  96,  71,  53,  19,  44,  77,  96, 102,  78,  52,
        45,  77,  92, 115, 127, 104,  81, 102, 126, 159, 176, 179, 151,
       130, 122, 142, 164, 177, 175, 165, 142, 115, 142, 165, 179, 175,
       158, 133, 114, 133, 162, 180, 172, 145, 126,  97, 123, 148, 170,
       178, 170, 142, 122, 142, 172, 175, 178, 176], dtype=int64)

In [18]:
#使用整数规划进行求解
def solveIP(families,min_occupancy,max_occupancy):
    #创建整数规划求解器CBC
    solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    #需要安排的家庭
    n_families = len(families)


    x = {}  # family_id 在第j天是否参观
    # 每一天有哪些家庭
    candidates = [[] for x in range(N_DAYS)]

    for i in families:  # family_id
        for j in DESIRED[i, :]:  # family_id的choice
            candidates[j].append(i)  # 在第j天，有第i个family参观
            # 定义变量[i][j] i代表family_id j代表第j天参观
            x[i, j] = solver.BoolVar('x[%i,%i]' % (i, j))
    # print(x)

    # 每天参观的人数
    daily_occupancy = [solver.Sum([x[i, j] * FAMILY_SIZE[i] for i in candidates[j]]) for j in range(N_DAYS)]

    # 每个家庭，在10个choice中出现的总数
    family_presence = [solver.Sum(x[i, j] for j in DESIRED[i, :]) for i in families]

    # 定义目标函数， preference_cost部分
    preference_cost = solver.Sum([pcost_mat[i, j] * x[i, j] for i in families for j in DESIRED[i, :]])

    # 满足preference_cost 最小
    solver.Minimize(preference_cost)



    # 每个家庭都在10个choice中出现1次
    for i in range(n_families):
        solver.Add(family_presence[i] == 1)

    # 每天访问人数约束
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j] >= min_occupancy[j])
        solver.Add(daily_occupancy[j] <= max_occupancy[j])

    result = solver.Solve()
    # 查看解
    temp = [(i, j) for i in families for j in DESIRED[i, :] if
            x[i, j].solution_value() > 0]

    # 得到参观日的安排
    df = pd.DataFrame(temp, columns=['family_id', 'day'])
    #print(solver.Objective().Value())
    return df


In [19]:
#没有安排的families_id
unassigned = unassigned_df.family_id.unique()
result = solveIP(unassigned,min_occupancy,max_occupancy)

In [20]:
result

,family_id,day
0,59,38
1,240,32
2,262,31
3,357,24
4,488,39
...,...,...
64,4869,59
65,4886,98
66,4912,17
67,4914,38


In [21]:
#将两部分计算的数据进行合并
df=pd.concat((assigned_df[['family_id', 'day']],result)).sort_values('family_id')
df

,family_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5068,4995,15
5069,4996,87
5070,4997,31
5071,4998,91


In [22]:
from numba import njit
#根据安排情况，计算安排的preference cost
@njit(fastmath=True)
def pcost(prediction):
    daily_occupancy = np.zeros(N_DAYS+1,dtype=np.int64)
    penalty = 0
    for (i,p) in enumerate(prediction):
        #计算家庭人数
        n = FAMILY_SIZE[i]
        #第i个家庭，第p天访问的cost
        penalty +=pcost_mat[i,p]
        #计算当天的人数
        daily_occupancy[p] +=n
    return penalty,daily_occupancy


#根据安排情况，计算安排的accounting cost
@njit(fastmath=True)
def acost(daily_occupancy):
    accounting_cost = 0
    num_out_of_range = 0
    daily_occupancy[-1] = daily_occupancy[-2]
    for day in range(N_DAYS):
        n_pl = daily_occupancy[day+1] #前一天
        n = daily_occupancy[day] #当天
        #如果超过了承载范围 则设置out of range
        num_out_of_range += (n >MAX_OCCUPANCY) or (n <MIN_OCCUPANCY)
        #计算accounting cost
        accounting_cost +=acost_mat[n,n_pl]
    return accounting_cost,num_out_of_range


@njit(fastmath=True)
def cost_function(prediction):
    #基于prediction，计算preference cost 和 accounting cost
    penalty,daily_occupancy = pcost(prediction)#统计preference cost 和 每天承载数量
    accounting_cost, num_out_of_range = acost(daily_occupancy) #根据每天承载数量，计算accounting cost
    final_score = penalty + accounting_cost +num_out_of_range*99999999
    return final_score



In [23]:
prediction = df.day.values
print(len(prediction))

5000


In [24]:
print(cost_function(prediction))

77544.53130943188


In [25]:
acost_mat.shape

(301, 301)

In [26]:
#将结果输出
def save_results(pred,filename):
    result = pd.DataFrame(range(N_FAMILY),columns=['fimily_id'])
    result['assigned_day'] = pred+1
    result.to_csv(filename,index=False)
    print(filename,'  saved')
    return  result

result = save_results(prediction,'submission.csv')

submission.csv   saved


In [27]:
#寻找更好的替代方案
def find_better(pred):
    fobs = np.argsort(FAMILY_SIZE)
    score = cost_function(pred)
    print('score：',score)
    original_score = np.inf #打擂法

    #如果找不到更新就退出
    while score < original_score:
        original_score = score

        for family_id in fobs:
            for pick in range(10):
                #得到family_id 在choice_pick的日期day
                day = DESIRED[family_id,pick]
                #该family的原有日期 oldvalue
                oldvalue = pred[family_id]
                #将原有的oldvalue 替换为现在的choice pick
                pred[family_id] = day
                #重新计算分数
                new_score = cost_function(pred)
                print('newscore:',new_score)
                print('score:',score)
                #如果比原来分数小，更新choice成功
                if new_score < score:
                    new_score = score
                else:#设置为原来的oldvalue
                    pred[family_id] = oldvalue
        print(score,end='\r')
    print(score)
    

In [28]:
new = prediction.copy()
find_better(new)

score： 77544.53130943188
newscore: 77544.53130943188
score: 77544.53130943188
newscore: 77573.37137344523
score: 77544.53130943188
newscore: 77598.52357409158
score: 77544.53130943188
newscore: 100077535.40478492
score: 77544.53130943188
newscore: 77826.88592960703
score: 77544.53130943188
newscore: 77737.70536518759
score: 77544.53130943188
newscore: 77856.35104137872
score: 77544.53130943188
newscore: 77905.23784583494
score: 77544.53130943188
newscore: 78014.93064026408
score: 77544.53130943188
newscore: 78513.50399935497
score: 77544.53130943188
newscore: 77544.53130943188
score: 77544.53130943188
newscore: 77606.41384791178
score: 77544.53130943188
newscore: 77580.49965843186
score: 77544.53130943188
newscore: 77613.55286868168
score: 77544.53130943188
newscore: 77789.18314715053
score: 77544.53130943188
newscore: 77761.1465785492
score: 77544.53130943188
newscore: 100077758.66225024
score: 77544.53130943188
newscore: 77895.80677082107
score: 77544.53130943188
newscore: 77994.8993

newscore: 77774.94543712685
score: 77544.53130943188
newscore: 77854.82136545303
score: 77544.53130943188
newscore: 77906.24593626396
score: 77544.53130943188
newscore: 77970.55748912464
score: 77544.53130943188
newscore: 78463.02066118746
score: 77544.53130943188
newscore: 77536.96298962388
score: 77544.53130943188
newscore: 77644.3905064378
score: 77544.53130943188
newscore: 77656.51203731104
score: 77544.53130943188
newscore: 77633.5630134705
score: 77544.53130943188
newscore: 77740.30311531879
score: 77544.53130943188
newscore: 77811.98186968922
score: 77544.53130943188
newscore: 77901.70324392544
score: 77544.53130943188
newscore: 77906.53992758718
score: 77544.53130943188
newscore: 78051.98677683779
score: 77544.53130943188
newscore: 78493.5346024579
score: 77544.53130943188
newscore: 77536.96298962388
score: 77544.53130943188
newscore: 77555.40058357487
score: 77544.53130943188
newscore: 77613.92282407786
score: 77544.53130943188
newscore: 77630.85480527663
score: 77544.53130943

newscore: 77875.77591229486
score: 77544.53130943188
newscore: 77977.06048549066
score: 77544.53130943188
newscore: 78509.10441333397
score: 77544.53130943188
newscore: 77536.96298962388
score: 77544.53130943188
newscore: 77652.88505797261
score: 77544.53130943188
newscore: 77686.55541829219
score: 77544.53130943188
newscore: 77695.2829691413
score: 77544.53130943188
newscore: 77804.60257470334
score: 77544.53130943188
newscore: 77786.01908482268
score: 77544.53130943188
newscore: 77957.8487163531
score: 77544.53130943188
newscore: 77971.61328205837
score: 77544.53130943188
newscore: 78066.271874685
score: 77544.53130943188
newscore: 78562.16669958613
score: 77544.53130943188
newscore: 77536.96298962388
score: 77544.53130943188
newscore: 4795551578040022.0
score: 77544.53130943188
newscore: 77615.27508092362
score: 77544.53130943188
newscore: 77642.99314059615
score: 77544.53130943188
newscore: 77751.29220979719
score: 77544.53130943188
newscore: 77779.57798357535
score: 77544.53130943

score: 77544.53130943188
newscore: 77657.9772108387
score: 77544.53130943188
newscore: 77727.27420035229
score: 77544.53130943188
newscore: 77748.1063693114
score: 77544.53130943188
newscore: 77844.68124578841
score: 77544.53130943188
newscore: 77880.98362742401
score: 77544.53130943188
newscore: 77985.42921148664
score: 77544.53130943188
newscore: 78512.82934445438
score: 77544.53130943188
newscore: 77537.78845697624
score: 77544.53130943188
newscore: 77613.75866612869
score: 77544.53130943188
newscore: 77620.96107907481
score: 77544.53130943188
newscore: 77665.7417901129
score: 77544.53130943188
newscore: 77765.16902311057
score: 77544.53130943188
newscore: 77785.95430044734
score: 77544.53130943188
newscore: 77883.69398343828
score: 77544.53130943188
newscore: 77886.85693104878
score: 77544.53130943188
newscore: 77957.44854044703
score: 77544.53130943188
newscore: 78517.35837763347
score: 77544.53130943188
newscore: 77537.78845697624
score: 77544.53130943188
newscore: 100077487.9232

score: 77544.53130943188
newscore: 77553.17040226817
score: 77544.53130943188
newscore: 77618.83985132682
score: 77544.53130943188
newscore: 77650.77981658818
score: 77544.53130943188
newscore: 77721.84668423189
score: 77544.53130943188
newscore: 77745.52269685766
score: 77544.53130943188
newscore: 77843.43174328309
score: 77544.53130943188
newscore: 77904.25123788115
score: 77544.53130943188
newscore: 77973.25765446646
score: 77544.53130943188
newscore: 78477.55737225852
score: 77544.53130943188
newscore: 77530.26879037819
score: 77544.53130943188
newscore: 77616.21892295696
score: 77544.53130943188
newscore: 77614.22811703313
score: 77544.53130943188
newscore: 77639.13795897196
score: 77544.53130943188
newscore: 77748.30020733786
score: 77544.53130943188
newscore: 77792.70818701149
score: 77544.53130943188
newscore: 77866.92876957578
score: 77544.53130943188
newscore: 77917.3382200099
score: 77544.53130943188
newscore: 78008.85619912679
score: 77544.53130943188
newscore: 100078392.80

...

score: 77544.53130943188
newscore: 200079606.6333088
score: 77544.53130943188
newscore: 77613.18415068903
score: 77544.53130943188
newscore: 77543.27629897105
score: 77544.53130943188
newscore: 100077645.55603361
score: 77544.53130943188
newscore: 77772.93049536886
score: 77544.53130943188
newscore: 100602793684890.67
score: 77544.53130943188
newscore: 77872.16390020156
score: 77544.53130943188
newscore: 78002.1274195371
score: 77544.53130943188
newscore: 78112.11695108682
score: 77544.53130943188
newscore: 78279.02809224717
score: 77544.53130943188
newscore: 79757.13757822842
score: 77544.53130943188
newscore: 77645.26508435473
score: 77544.53130943188
newscore: 77543.27629897105
score: 77544.53130943188
newscore: 77796.49601750352
score: 77544.53130943188
newscore: 77849.49441215779
score: 77544.53130943188
newscore: 77902.56105721035
score: 77544.53130943188
newscore: 77949.62972838321
score: 77544.53130943188
newscore: 78156.91548568777
score: 77544.53130943188
newscore: 78184.3126

score: 77544.53130943188
newscore: 77543.27629897105
score: 77544.53130943188
newscore: 77637.73085681703
score: 77544.53130943188
newscore: 499861808474551.8
score: 77544.53130943188
newscore: 77678.72470928525
score: 77544.53130943188
newscore: 77900.13600309019
score: 77544.53130943188
newscore: 77856.09177186614
score: 77544.53130943188
newscore: 78029.22166484888
score: 77544.53130943188
newscore: 78156.27690923808
score: 77544.53130943188
newscore: 78194.45217906778
score: 77544.53130943188
newscore: 100079855.86588672
score: 77544.53130943188
newscore: 77556.36716779158
score: 77544.53130943188
newscore: 100077498.20894088
score: 77544.53130943188
newscore: 77543.27629897105
score: 77544.53130943188
newscore: 77765.80747153971
score: 77544.53130943188
newscore: 77740.71142407026
score: 77544.53130943188
newscore: 77883.84697577223
score: 77544.53130943188
newscore: 78028.42346545294
score: 77544.53130943188
newscore: 78039.05582912182
score: 77544.53130943188
newscore: 499861808

score: 77544.53130943188
newscore: 2639282590754154.0
score: 77544.53130943188
newscore: 77951.2545247605
score: 77544.53130943188
newscore: 78016.81754396233
score: 77544.53130943188
newscore: 78191.87034608335
score: 77544.53130943188
newscore: 78263.20923489807
score: 77544.53130943188
newscore: 80003.34705666234
score: 77544.53130943188
newscore: 77519.74422058843
score: 77544.53130943188
newscore: 77543.27629897105
score: 77544.53130943188
newscore: 3817787231620972.0
score: 77544.53130943188
newscore: 77718.23369878571
score: 77544.53130943188
newscore: 77788.36004191128
score: 77544.53130943188
newscore: 77787.85885524425
score: 77544.53130943188
newscore: 77900.07804449623
score: 77544.53130943188
newscore: 78044.9780916242
score: 77544.53130943188
newscore: 78119.78269958653
score: 77544.53130943188
newscore: 79893.2138637601
score: 77544.53130943188
newscore: 77543.27629897105
score: 77544.53130943188
newscore: 77706.37463373602
score: 77544.53130943188
newscore: 77776.817784

newscore: 77717.19532694126
score: 77544.53130943188
newscore: 77811.97627035837
score: 77544.53130943188
newscore: 77926.91319189877
score: 77544.53130943188
newscore: 100077853.31466125
score: 77544.53130943188
newscore: 77975.69931817379
score: 77544.53130943188
newscore: 78110.12152802697
score: 77544.53130943188
newscore: 78395.09136362969
score: 77544.53130943188
newscore: 79937.26486042267
score: 77544.53130943188
newscore: 77543.27629897105
score: 77544.53130943188
newscore: 77612.08139443668
score: 77544.53130943188
newscore: 100077613.04079548
score: 77544.53130943188
newscore: 77752.06763933209
score: 77544.53130943188
newscore: 77789.75616015357
score: 77544.53130943188
newscore: 77944.43531614848
score: 77544.53130943188
newscore: 78016.8532464652
score: 77544.53130943188
newscore: 78187.54434991107
score: 77544.53130943188
newscore: 78328.4146543077
score: 77544.53130943188
newscore: 79875.13982163489
score: 77544.53130943188
newscore: 77543.27629897105
score: 77544.53130

score: 77544.53130943188
newscore: 100078066.20106739
score: 77544.53130943188
newscore: 78202.42695254413
score: 77544.53130943188
newscore: 79940.01351837204
score: 77544.53130943188
newscore: 77559.77382472376
score: 77544.53130943188
newscore: 77542.9379410004
score: 77544.53130943188
newscore: 100077653.02897829
score: 77544.53130943188
newscore: 77812.42349895519
score: 77544.53130943188
newscore: 77880.08270093094
score: 77544.53130943188
newscore: 77975.22380200763
score: 77544.53130943188
newscore: 78142.20412569187
score: 77544.53130943188
newscore: 89919747992768.45
score: 77544.53130943188
newscore: 78248.84837527563
score: 77544.53130943188
newscore: 80108.80568067606
score: 77544.53130943188
newscore: 77542.9379410004
score: 77544.53130943188
newscore: 77634.20925892425
score: 77544.53130943188
newscore: 77723.57149188567
score: 77544.53130943188
newscore: 77852.33482626121
score: 77544.53130943188
newscore: 77995.79909676277
score: 77544.53130943188
newscore: 78015.27704

In [29]:
save_results(new,'submission.csv')

submission.csv   saved


,fimily_id,assigned_day
0,0,52
1,1,26
2,2,100
3,3,2
4,4,53
...,...,...
4995,4995,16
4996,4996,88
4997,4997,32
4998,4998,92
